# Part D
#### Grading Criteria
Repeat part B but use a neural network with the following instead:
- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [3]:
# perform installs
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install scikit-learn

In [4]:
# imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

import warnings
warnings.simplefilter('ignore', FutureWarning)

2025-03-02 05:33:08.178588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-02 05:33:08.225223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Get concrete dataset
filepath='https://cocl.us/concrete_data'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
# check how many data points we have
concrete_data.shape

(1030, 9)

In [7]:
# look at dataset info
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [8]:
# make sure we don't have null values
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [9]:
# Split dataset into predictors (all columns except Strength) and target (Strength column)
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [10]:
# normalize predictors
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

In [12]:
# function to build model
def build_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    # for part d, model has 3 hidden layers, all with 10 modes and relu
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error') # Use adam and mse
    return model

In [13]:
# build model
model = build_model()

In [14]:
mse_list = []

In [17]:
def test_mse():
    # split into train and test sets and hold 30% of data for testing
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
    # train model with 50 epochs
    model.fit(X_train, y_train, epochs=50, verbose=2)
    y_pred = model.predict(X_test)  #eval model on test data
    mse = mean_squared_error(y_test, y_pred)  # compute mse
    mse_list.append(mse)  #add to list of mses

In [18]:
# repeat 50 times
num_iterations = 50
for i in range(num_iterations):
    test_mse()

Epoch 1/50
23/23 - 1s - 51ms/step - loss: 1608.2933
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1570.1125
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1526.7855
Epoch 4/50
23/23 - 0s - 6ms/step - loss: 1469.8599
Epoch 5/50
23/23 - 0s - 5ms/step - loss: 1390.2140
Epoch 6/50
23/23 - 0s - 4ms/step - loss: 1281.3939
Epoch 7/50
23/23 - 0s - 4ms/step - loss: 1135.0624
Epoch 8/50
23/23 - 0s - 5ms/step - loss: 943.3547
Epoch 9/50
23/23 - 0s - 4ms/step - loss: 730.6497
Epoch 10/50
23/23 - 0s - 4ms/step - loss: 522.0101
Epoch 11/50
23/23 - 0s - 4ms/step - loss: 370.8587
Epoch 12/50
23/23 - 0s - 4ms/step - loss: 300.2736
Epoch 13/50
23/23 - 0s - 5ms/step - loss: 266.1520
Epoch 14/50
23/23 - 0s - 4ms/step - loss: 245.7483
Epoch 15/50
23/23 - 0s - 5ms/step - loss: 230.9887
Epoch 16/50
23/23 - 0s - 4ms/step - loss: 219.8221
Epoch 17/50
23/23 - 0s - 4ms/step - loss: 210.9870
Epoch 18/50
23/23 - 0s - 4ms/step - loss: 204.1456
Epoch 19/50
23/23 - 0s - 4ms/step - loss: 198.2690
Epoch 20/50
23/23 - 0s - 4ms/ste

In [19]:
# Report the mean and the standard deviation of the mean squared errors.
average_mse = np.mean(mse_list)
print(f"Average Mean Squared Error over {num_iterations} iterations: {average_mse}")
std_mse = np.std(mse_list)
print(f"Mean Squared Error Standard Deviation over {num_iterations} iterations: {std_mse}")

Average Mean Squared Error over 50 iterations: 40.90905237340135
Mean Squared Error Standard Deviation over 50 iterations: 12.68507341077861


In [1]:
# Part A metrics:
# Average Mean Squared Error over 50 iterations: 66.2795128282057
# Mean Squared Error Standard Deviation over 50 iterations: 60.89369286469975

# Part B metrics
# Average Mean Squared Error over 50 iterations: 51.930608871813966
# Mean Squared Error Standard Deviation over 50 iterations: 36.85541987092113

# Part C metrics
# Average Mean Squared Error over 50 iterations: 40.26975296753347
# Mean Squared Error Standard Deviation over 50 iterations: 20.859128230630006

# How does the mean of the mean squared errors compare to that from Step B?
For part B, the mean of the mean squared errors was 51.930608871813966, and the standard deviation was 36.85541987092113.
For part D, the mean of the mean squared errors was 40.90905237340135, and the standard deviation was 12.68507341077861.
This model performed better than the model in part B, indicating that the addition of two hidden layers helped. Compared to part C, this model's mean of MSEs was slightly higher, but the standard deviation was lower. I would suspect that if we combined parts C and D (using 100 epoch and 3 hidden layers), the model's performance would further improve.